# Beers With Brute Force

    by Brian Groce, Diane Scherpereel, Roger Albarran

In [1]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [31]:
##    ASK MARK ABOUT RELATIVE FILE PATH   ****

# Pull the beer reviews from the csv file. Read it and put it into a pandas dataframe.
# csv file source: Kaggle.com  beer_reviews.csv
reviews_csv = "/Users/dianeshomefolder/Documents/git/Beers_With_Brute_Force/Beer_reviews.csv"
reviews_df = pd.read_csv(reviews_csv)
reviews_df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [10]:
# Pull the brewery contact information from the csv file. Read it and put it into a pandas dataframe.
# csv file source: Kaggle.com  7160_1.csv
contact_csv = "/Users/dianeshomefolder/Documents/git/Beers_With_Brute_Force/7160_1.csv"
contact_df = pd.read_csv(contact_csv)
contact_df.head()

,address,categories,city,country,key,lat,long,name,phones,postalCode,province,websites
0,407 Radam Ln,brewery,Austin,US,us/tx/austin/407radamln,NaN,NaN,(512) Brewing Co,512-707-2337,78745,TX,NaN
1,1135 N W Galveston Ave,Brewery and Bar,Bend,US,us/or/bend/1135nwgalvestonave,NaN,NaN,10 Barrel Brewing Co.,(541) 585-1007,97703,OR,10barrel.com
2,830 W Bannock St,"brewery, Restaurant, Pub, Brewery, pub, Brewer...",Boise,US,us/id/boise/830wbannockst,43.617711,-116.202883,10 Barrel Brewing Co.,(208) 344-5870,83702,ID,10barrel.com
3,1022 Texan Trl,"Wine Tours & Tastings, Food & Drink, Tours, Br...",Grapevine,US,us/tx/grapevine/1022texantrl,32.938385,-97.064340,10 Gallon Tours,817-403-1832,76051,TX,http://www.10gallontours.com/
4,Po Box 135,"American Restaurants, Bar & Grill Restaurants,...",Villard,US,us/mn/villard/pobox135,NaN,NaN,10 Mile Tavern,320-554-2939,56385,MN,http://www.10miletavern.com


### Transform Beer Reviews DataFrame

In [18]:
# Create a filtered dataframe from specific columns
reviews_cols = ["beer_name", "beer_style", "brewery_name", "review_overall", "review_aroma", "review_palate", "review_taste"]
reviews_transformed = reviews_df[reviews_cols].copy()
reviews_transformed.head()


,beer_name,beer_style,brewery_name,review_overall,review_aroma,review_palate,review_taste
0,Sausa Weizen,Hefeweizen,Vecchio Birraio,1.5,2.0,1.5,1.5
1,Red Moon,English Strong Ale,Vecchio Birraio,3.0,2.5,3.0,3.0
2,Black Horse Black Beer,Foreign / Export Stout,Vecchio Birraio,3.0,2.5,3.0,3.0
3,Sausa Pils,German Pilsener,Vecchio Birraio,3.0,3.0,2.5,3.0
4,Cauldron DIPA,American Double / Imperial IPA,Caldera Brewing Company,4.0,4.5,4.0,4.5


In [38]:
# Set index as name column
reviews_transformed.set_index('brewery_name')

,beer_name,beer_style,review_overall,review_aroma,review_palate,review_taste
brewery_name,,,,,,
Vecchio Birraio,Sausa Weizen,Hefeweizen,1.5,2.0,1.5,1.5
Vecchio Birraio,Red Moon,English Strong Ale,3.0,2.5,3.0,3.0
Vecchio Birraio,Black Horse Black Beer,Foreign / Export Stout,3.0,2.5,3.0,3.0
Vecchio Birraio,Sausa Pils,German Pilsener,3.0,3.0,2.5,3.0
Caldera Brewing Company,Cauldron DIPA,American Double / Imperial IPA,4.0,4.5,4.0,4.5
Caldera Brewing Company,Caldera Ginger Beer,Herbed / Spiced Beer,3.0,3.5,3.0,3.5
Amstel Brouwerij B. V.,Amstel Light,Light Lager,3.0,2.0,2.5,2.5
Caldera Brewing Company,Caldera Oatmeal Stout,Oatmeal Stout,3.0,3.0,3.0,3.0
Caldera Brewing Company,Caldera OBF 15,American Pale Lager,4.0,3.0,4.0,4.0


In [68]:

# Create an average_review column for the average of all 4 reviews
avg_review = reviews_transformed
avg_review["average_review"] = avg_review.mean(numeric_only=True, axis=1)
avg_review

,beer_style,brewery_name,review_overall,review_aroma,review_palate,review_taste,average_review
beer_name,,,,,,,
Dominion Saison,Saison / Farmhouse Ale,Old Dominion Brewing Company,5.0,5.0,5.0,5.0,5.0
Randy's Panhead Porter,English Porter,The Ship Inn,5.0,5.0,5.0,5.0,5.0
Rail Spur Double IPA Bourbon Barrel Aged,American Double / Imperial IPA,Intuition Ale Works,5.0,5.0,5.0,5.0,5.0
Tricerahops Double IPA,American Double / Imperial IPA,Ninkasi Brewing Company,5.0,5.0,5.0,5.0,5.0
Jacobins Framboise,Lambic - Fruit,Brouwerij Bockor N.V.,5.0,5.0,5.0,5.0,5.0
Killer Bee,American Pale Ale (APA),The Ship Inn,5.0,5.0,5.0,5.0,5.0
Great Lakes Truth Justice And The American Ale,American Pale Ale (APA),Great Lakes Brewing Company,5.0,5.0,5.0,5.0,5.0
Eichbaum Pilsener,German Pilsener,Privatbrauerei Eichbaum GmbH & Co. KG,5.0,5.0,5.0,5.0,5.0
Holiday Cheer,American Porter,New Old Lompoc Brewery,5.0,5.0,5.0,5.0,5.0


### Transform Brewery Contact Information DataFrame

In [20]:
# Create a filtered dataframe from specific columns
contact_cols = ["name", "address", "city", "province", "postalCode", "phones"]
contact_transformed = contact_df[contact_cols].copy()
contact_transformed.head()

,name,address,city,province,postalCode,phones
0,(512) Brewing Co,407 Radam Ln,Austin,TX,78745,512-707-2337
1,10 Barrel Brewing Co.,1135 N W Galveston Ave,Bend,OR,97703,(541) 585-1007
2,10 Barrel Brewing Co.,830 W Bannock St,Boise,ID,83702,(208) 344-5870
3,10 Gallon Tours,1022 Texan Trl,Grapevine,TX,76051,817-403-1832
4,10 Mile Tavern,Po Box 135,Villard,MN,56385,320-554-2939


### Rename some of the columns and clean the data in the Brewery Contact dataframe

In [37]:
# Rename some of the columns
contact_transformed_renamed = contact_transformed.rename(columns={"province": "state",
                                                         "postalCode": "zip",
                                                         "phones": "phone"})
# Replace 'NaN' in phone number column with blank
contact_transformed_renamed.fillna('', inplace=True)
contact_transformed_renamed.head()

,name,address,city,state,zip,phone
0,(512) Brewing Co,407 Radam Ln,Austin,TX,78745,512-707-2337
1,10 Barrel Brewing Co.,1135 N W Galveston Ave,Bend,OR,97703,(541) 585-1007
2,10 Barrel Brewing Co.,830 W Bannock St,Boise,ID,83702,(208) 344-5870
3,10 Gallon Tours,1022 Texan Trl,Grapevine,TX,76051,817-403-1832
4,10 Mile Tavern,Po Box 135,Villard,MN,56385,320-554-2939


In [36]:
# Set index as name column
contact_transformed_renamed.set_index('name') #, inplace=True)

,address,city,state,zip,phone
name,,,,,
(512) Brewing Co,407 Radam Ln,Austin,TX,78745,512-707-2337
10 Barrel Brewing Co.,1135 N W Galveston Ave,Bend,OR,97703,(541) 585-1007
10 Barrel Brewing Co.,830 W Bannock St,Boise,ID,83702,(208) 344-5870
10 Gallon Tours,1022 Texan Trl,Grapevine,TX,76051,817-403-1832
10 Mile Tavern,Po Box 135,Villard,MN,56385,320-554-2939
101 Cider House,31111 Via Colinas,Westlake Village,CA,91362,818-851-9057
101 North Brewing Company,1304 Scott St,Petaluma,CA,94954,707-778-8384
105 West Brewery,1043 Park St,Castle Rock,CO,80109,
10-speed Coffee Bar,1412 13th St,Hood River,OR,97031,541-386-3165


### Create database connection

In [41]:
connection_string = "postgres:postgres@localhost:5432/beer_reviews"
engine = create_engine(f'postgresql://{connection_string}')

In [42]:

# Confirm tables   - in PGAmin database name is beer_reviews
engine.table_names()

['contact', 'reviews']

### Load DataFrames into database

In [45]:
reviews_transformed.to_sql(name='reviews', con=engine, if_exists='append', index=False)

In [46]:
contact_transformed_renamed.to_sql(name='contact', con=engine, if_exists='append', index=False)

In [69]:
avg_review.to_csv(r'/Users/dianeshomefolder/Documents/git/Beers_With_Brute_Force/average_review.csv')